#----------------------------------------------------------------------------#
#              KICT-RAIN-AI using Grided Rainfall data                       #
#   Ver.1 revised by Seong-Sim Yoon  Oct.2025                                #
#   Copyright : KOREA INSTITUTE of CIVIL ENGINEERING and BUILDING TECHNOLOGY #
#----------------------------------------------------------------------------#
#Input : ASCII File(radar rainfall)
#Input Data Coordinate  : EPSG5186
#Resoultion : 1km, every 10min
#units : mm/hr
#Ouput : ASCII File(grid rainfall)

In [1]:
from google.colab import drive
# 구글드라이브 사전학습된 모델 KICT-RAIN-AI_v3 (.tflite) 불러오기
# 코랩 가상 드라이브에 저장
# Create directory first
!mkdir -p /content/models

# Download pretrained QPF models with proper gdown syntax
!gdown 1uUycfjU-jZro74Rob_4K2EAVHoh-S1VB -O /content/models/model-best_fcst_10min_re.tflite
!gdown 1rKIZZUCbmaLZ5cT0XYtFK_JIMKp2_QZM -O /content/models/model-best_fcst_20min_re.tflite
!gdown 1-lBK-5tTttBjaJP-KdsJ9RpEVIvUHvtu -O /content/models/model-best_fcst_30min_re.tflite
!gdown 1repj4aKb_fMg2RouosvZhQLJaKRWdbI6 -O /content/models/model-best_fcst_40min_re.tflite
!gdown 1mCcejAfxIFvCrJM3fIoX_mN-590B5n_0 -O /content/models/model-best_fcst_50min_re.tflite
!gdown 1WvJVS3RdzZ7frzn3YPM7U5pvWuYffhpk -O /content/models/model-best_fcst_60min_re.tflite
!gdown 1-cBLlUPwqTvEdAIrJ2eq46nqrEf4wLR3 -O /content/models/model-best_fcst_70min_re.tflite
!gdown 1K6VU-uFubH2C8wE4EvRozboTmU7IFHUp -O /content/models/model-best_fcst_80min_re.tflite
!gdown 1tvwKiYbGaocb6_6M-JCBG99yOXa_OwSe -O /content/models/model-best_fcst_90min_re.tflite
!gdown 1S0ZlxGipRq5tnSM5UPPFZ15_iv7Y143a -O /content/models/model-best_fcst_100min_re.tflite
!gdown 1EWcTbOrIOQq1y9CKq13QkA7EC0oibqmq -O /content/models/model-best_fcst_110min_re.tflite
!gdown 13gEwH8mtak5ZTYLslaWeMx_gUxmuu1SD -O /content/models/model-best_fcst_120min_re.tflite
!gdown 1I2n32WEPm_TCyWEdzeUJwIi5xI_eSmd9 -O /content/models/model-best_fcst_130min_re.tflite
!gdown 1w5S8C3_jPrBlFu_M6Xogxp8lk-gt8SmY -O /content/models/model-best_fcst_140min_re.tflite
!gdown 1lFaZnDnNwpM3C-0gBA4P_1vXqHCDwW7s -O /content/models/model-best_fcst_150min_re.tflite
!gdown 1evUSHjZauC94Wn4MmffHf4FgDASIj-MC -O /content/models/model-best_fcst_160min_re.tflite
!gdown 1LvF2mcMwZEeLimMqkYrpgk44b1Zw1wKi -O /content/models/model-best_fcst_170min_re.tflite
!gdown 1zfO7YjO6CQyB_utvzAPoya3-1fgv6xCn -O /content/models/model-best_fcst_180min_re.tflite
print("All models downloaded successfully!")

Downloading...
From: https://drive.google.com/uc?id=1uUycfjU-jZro74Rob_4K2EAVHoh-S1VB
To: /content/models/model-best_fcst_10min_re.tflite
100% 7.86M/7.86M [00:00<00:00, 176MB/s]
Downloading...
From: https://drive.google.com/uc?id=1rKIZZUCbmaLZ5cT0XYtFK_JIMKp2_QZM
To: /content/models/model-best_fcst_20min_re.tflite
100% 7.86M/7.86M [00:00<00:00, 27.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-lBK-5tTttBjaJP-KdsJ9RpEVIvUHvtu
To: /content/models/model-best_fcst_30min_re.tflite
100% 7.86M/7.86M [00:00<00:00, 30.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1repj4aKb_fMg2RouosvZhQLJaKRWdbI6
To: /content/models/model-best_fcst_40min_re.tflite
100% 7.86M/7.86M [00:00<00:00, 30.2MB/s]
Downloading...
From: https://drive.google.com/uc?id=1mCcejAfxIFvCrJM3fIoX_mN-590B5n_0
To: /content/models/model-best_fcst_50min_re.tflite
100% 7.86M/7.86M [00:00<00:00, 21.6MB/s]
Downloading...
From: https://drive.google.com/uc?id=1WvJVS3RdzZ7frzn3YPM7U5pvWuYffhpk
To: /content/models/

In [2]:
# Input data 불러오기
# 코랩 가상 드라이브에 저장
!gdown https://drive.google.com/uc?id=1b2Thq96CqzYzaT34w1OGZ-vPV9-q3XwU #INPUT_ASC_20220808.zip 파일
!unzip -qq '/content/INPUT_ASC_20220808.zip' -d '/content/data'

Downloading...
From: https://drive.google.com/uc?id=1b2Thq96CqzYzaT34w1OGZ-vPV9-q3XwU
To: /content/INPUT_ASC_20220808.zip
100% 8.51M/8.51M [00:00<00:00, 226MB/s]


In [4]:
import os
from tqdm import tqdm
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_absolute_error
from sklearn.utils import shuffle
from tqdm import tqdm
import time
import glob
import tensorflow as tf
import cv2
import gc
from PIL import Image
from keras.models import load_model
import datetime
import keras
import keras.backend as K
from keras.layers import *
from keras.models import Model
from keras.losses import mean_absolute_error
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

# Use the new location for ImageDataGenerator
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.optimizers import Nadam
from keras.metrics import AUC

print("All imports successful!")

All imports successful!


In [5]:
from keras import backend as K
import warnings
warnings.simplefilter("ignore")
# Metrics
def mae255(y_true, y_pred):
    return 255 * mean_absolute_error(y_true, y_pred)

# f1 score 계산
def Rain(array):
    R=(array*255.)
    return R

def CSI_m(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    y_true = y_true.reshape(1, -1)[0]
    y_pred = y_pred.reshape(1, -1)[0]
    remove_NAs = y_true > 0
    y_true = np.where(y_true[remove_NAs] >= 0.1, 1, 0)
    y_pred = np.where(y_pred[remove_NAs] >= 0.1, 1, 0)
    right=np.sum(y_true * y_pred == 1)
    #print(right,np.sum(y_pred),np.sum(y_true),right)
    CSI = right/(np.sum(y_pred)+np.sum(y_true)-right+1e-07)
    return CSI

def CSI_custom(y_true, y_pred):
    score = tf.py_function(func=CSI_m, inp=[y_true, y_pred], Tout=tf.float32,  name='CSI_custom')
    return score

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

def mae_custom(y_true, y_pred):
    y_pred=tf.convert_to_tensor(y_pred)
    y_true=tf.convert_to_tensor(y_true)
    y_true = Rain(y_true)
    y_pred = Rain(y_pred)
    thr1 = K.greater(y_true,0.1)
    loss1 =K.mean(K.abs(y_true[thr1] - y_pred[thr1]))
    arr=[loss1]
    value_not_nan = tf.dtypes.cast(tf.math.logical_not(tf.math.is_nan(arr)), dtype=tf.float32)
    loss0=tf.math.multiply_no_nan(arr, value_not_nan)
    loss=tf.convert_to_tensor(K.sum(loss0))
    return loss


#특정시점 예측

In [6]:
# 예측 시점 입력
!mkdir /content/QPF
date_nm='202208082200' #날짜 수정
run_date=date_nm[2:12]
print(run_date)

2208082200


In [7]:
# READ INPUT DATA (PAST 4 data)
def read_ASC(timestamp):
#    print(timestams)
    YY = timestamp[0:2]
    MM = timestamp[2:4]
    DD = timestamp[4:6]
    HH = timestamp[6:8]
    MN = timestamp[8:10]
    RDR_time = timestamp[0:10]
    YYYY = '20'+YY
    print(RDR_time)
    path='/content/data'
    filename = path+'/RDR_CMP_HSP_PUB_20'+RDR_time+'-525x625-1km.asc'
    data = np.zeros([625,525], np.float32) #ASCII랑배열 반대
    data = np.loadtxt(filename,skiprows=6)
    data[data < 0] = 0.
    attr = datetime.datetime(int(YYYY),int(MM),int(DD),int(HH),int(MN))
    return data, attr

def qpe_data(filetime):
    YY = filetime[0:2]
    MM = filetime[2:4]
    DD = filetime[4:6]
    HH = filetime[6:8]
    MN = filetime[8:10]
    YYYY = '20'+YY
    latest_datetime = datetime.datetime(int(YYYY),int(MM),int(DD),int(HH),int(MN))
    #print(latest_datetime)
    list_for_qpf = [ts.strftime("%y%m%d%H%M") for ts in
    [latest_datetime - datetime.timedelta(minutes=t) for t in [30, 20, 10, 0]]]
    input_scans = np.array([read_ASC(ts)[0] for ts in list_for_qpf])
    qpe_scans = np.concatenate([input_scans], axis=0)
    print(qpe_scans.shape)
    return qpe_scans

# KICT-RAIN-AI_ver3 (2025년 10월 2일 갱신)
기법: 매 선행 예측시점별 U-net을 최적화, 1개의 모델이 1개의 예측강우장 생성

갱신정보: 기상청 레이더 관측강우 자료를 활용한 KICT-RAIN-AI_ver2 모델 전이학습 수행

사전학습된 예측강우 모델: model-best_fcst_10min_re.tflite~model-best_fcst_180min_re.tflite, 18개

In [8]:
import numpy as np
from tensorflow.lite.python import interpreter as interpreter_wrapper

def data_preprocessing(X):
   # 0. Right shape for batch
   X = X[np.newaxis, ::, ::, ::]
   return X

def data_postprocessing(nwcst):
    # 0. Squeeze empty dimensions
    nwcst = np.squeeze(np.array(nwcst))
    nwcst = nwcst
    nwcst = np.where(nwcst>0, nwcst, 0)
    nwcst = nwcst[7:632, 9:534]
    return nwcst

def prediction(model_instance, input_data):
    input_data = data_preprocessing(input_data)
    print(input_data.shape)
    pred = model_instance.predict(input_data)
    print(pred.shape)
    nwcst = np.squeeze(pred)  # np.transpose() 대신 np.squeeze()를 사용하세요.
    return nwcst

def save_as_ascii_grid(data, output_file):
    """
    2D NumPy array를 ASCII grid 파일로 저장하는 함수
    :param data: 2D NumPy array (격자 자료)
    :param output_file: 저장할 파일 경로 및 이름 (예: 'output.asc')
    """
    #EPSG 5186 기준
    nrows, ncols = data.shape
    header = f"ncols {ncols}\n"
    header += f"nrows {nrows}\n"
    header += "xllcorner -31000\n"
    header += "yllcorner 123000\n"
    header += "cellsize 1000.0\n"  # 셀 크기
    header += "NODATA_value -9999.0\n"  # 누락된 데이터를 나타내는 값

    np.savetxt(output_file, data, fmt="%.2f", header=header, comments="", delimiter=" ")


def prediction_tflite(feature):
    qpf_all_each = np.empty((625, 525, 0))
    for j in range(10, 190, 10):
    # 이전 코드에서 필요한 설정과 모델 로딩을 진행한 후
        pre_model = f'/content/models/model-best_fcst_{j}min_re.tflite'
        interpreter = interpreter_wrapper.Interpreter(model_path=pre_model)
        interpreter.allocate_tensors()

        # 입력 텐서 정보 가져오기
        input_details = interpreter.get_input_details()
        output_details = interpreter.get_output_details()
        input_shape = input_details[0]['shape']

        # 입력 데이터 변환 및 설정
        feature = np.array(feature/255., dtype=np.float32)  # FLOAT64에서 FLOAT32로 변환
        interpreter.set_tensor(input_details[0]['index'], feature)
        # 모델 실행
        interpreter.invoke()

        # 출력 데이터 가져오기
        nwcst = interpreter.get_tensor(output_details[0]['index'])
        nwcst = data_postprocessing(nwcst)*255.
        # 예측하기
        qpf_np = nwcst

        #매 예측시점 자료를 ASCII GRID 형식으로 저장(EPSG518 기준)
        output_file_path = '/content/QPF/QPF_v3_'+date_nm+'-'+str(j)+'.asc'
        save_as_ascii_grid(qpf_np, output_file_path)
        #이미지 처리를 위한 3차원 배열 예측강우 생성
        qpf_np = np.expand_dims(qpf_np, axis=-1)  # 차원을 늘리세요.
        qpf_all_each = np.concatenate([qpf_all_each, qpf_np], axis=-1)
    return qpf_np, qpf_all_each
#RUN
dataset0=qpe_data(run_date)
dataset = np.transpose(dataset0,(1,2,0))
dataset = np.pad(dataset, [(7, 8), (9, 10), (0, 0)], mode='constant', constant_values=0)
feature0 = dataset[:, :, :]
feature = np.where(feature0 < 0., 0, feature0)
feature=data_preprocessing(feature)
print(feature.shape)

qpf_np, qpf_all_each=prediction_tflite(feature)

2208082130
2208082140
2208082150
2208082200
(4, 625, 525)
(1, 640, 544, 4)
